In [4]:
import urllib.request

In [5]:
!mkdir TSPLIB_DATA

mkdir: TSPLIB_DATA: File exists


In [6]:
TSPLIB_DATA="./TSPLIB_DATA/"

# Download sample TSLIB instances

In [9]:
def download_tsplib_instance(instance_name="a280.tsp"):
    
    base_url = "https://raw.githubusercontent.com/yafeunteun/tsplib/master/"
    complete_url = base_url + instance_name
    output_filepath = TSPLIB_DATA + instance_name
    urllib.request.urlretrieve(complete_url, output_filepath)

In [10]:
download_tsplib_instance()

In [13]:
!ls TSPLIB_DATA
!head TSPLIB_DATA/a280.tsp

a280.tsp
NAME : a280
COMMENT : drilling problem (Ludwig)
TYPE : TSP
DIMENSION: 280
EDGE_WEIGHT_TYPE : EUC_2D
NODE_COORD_SECTION
  1 288 149
  2 288 129
  3 270 133
  4 256 141


# Read TSPLIB

In [54]:
def read_tsplib(filepath_to_tsplib_file):
    coordinates = [] 
    dim = 0
    start_read_coords = False
    with open(filepath_to_tsplib_file, "r") as f:
        line = f.readline().strip() or "EOF"
        while not line.startswith("EOF"):
            if start_read_coords:
                _id, x, y = (float(x) for x in line.split())
                coordinates.append((x,y))
            if line.startswith("DIMENSION"):
                dim = int(line.split(':')[1].strip())
            if line.startswith("NODE_COORD_SECTION"):
                start_read_coords = True
            line = f.readline().strip()
    assert len(coordinates) == dim, "Number of coordinates doesn't match the dimension parameter in file."
    
    return coordinates

In [55]:
coords = read_tsplib(TSPLIB_DATA + 'a280.tsp')

In [56]:
coords[:5]

[(288.0, 149.0),
 (288.0, 129.0),
 (270.0, 133.0),
 (256.0, 141.0),
 (256.0, 157.0)]